**Cqlengine.models.Model**

Columns in your models map to columns in your CQL table. You define CQL columns by defining column attributes on your model classes. For a model to be valid it needs at least one primary key column and one non-primary key column. Just as in CQL, the order you define your columns in is important, and is the same order they are defined in on a model’s corresponding table.

To sync the models to the database, you may do the following*:

In [ ]:
from cassandra.cqlengine import columns
from cassandra.cqlengine.models import Model
from cassandra.cqlengine.management import sync_table

class Person(Model):
    id = columns.UUID(primary_key=True)
    first_name  = columns.Text()
    last_name = columns.Text()
    
    
class Comment(Model):
    photo_id = columns.UUID(primary_key=True)
    comment_id = columns.TimeUUID(primary_key=True, clustering_order="DESC")
    comment = columns.Text()
    
    
sync_table(Person)
sync_table(Comment)

**Model validation**

Similar to: https://sqlmodel.tiangolo.com/ - but without the ability to enforce data types outside of session event hooks

Each time you save a model instance in cqlengine, the data in the model is validated against the schema you’ve defined for your model. Most of the validation is fairly straightforward, it basically checks that you’re not trying to do something like save text into an integer column, and it enforces the required flag set on column definitions. It also performs any transformations needed to save the data properly.

However, there are often additional constraints or transformations you want to impose on your data, beyond simply making sure that Cassandra won’t complain when you try to insert it. To define additional validation on a model, extend the model’s validation method:

In [ ]:
class Member(Model):
    person_id = UUID(primary_key=True)
    name = Text(required=True)

    def validate(self):
        super(Member, self).validate()
        if self.name == 'jon':
            raise ValidationError('no jon\'s allowed')